# 1. Вытаскиваем ссылки, которые накачали

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pickle

# модуль с мелкими функциями для части информации
from cian_info import *

In [ ]:
with open('hrefs_total.pickle', 'rb') as f:
    hrefs_total = pickle.load(f)
    
len(hrefs_total)

In [ ]:
type(hrefs_total), len(set(hrefs_total))

### Чтобы включать ТОР

In [ ]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [ ]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

# 2. Качаем основную информацию по квартирам

Объединяем всё в одну функцию! 

In [ ]:
def infa_download(soup):
    infa_dict = {}
    infa_dict.update(get_general_information(soup))
    infa_dict.update(get_about(soup))
    infa_dict.update(get_price(soup))
    infa_dict.update(get_description(soup))
    infa_dict.update(get_square_info(soup))
    infa_dict.update(get_address(soup))
    infa_dict.update(get_metro(soup))
    infa_dict.update(get_title(soup))
    infa_dict.update(get_coord(soup))
    return infa_dict

Основной цикл: 

In [ ]:
all_infa = [ ] # для данных

In [ ]:
import time
from requests.exceptions import ConnectionError

for url in tqdm_notebook(hrefs_total):  # качаю для первого десятка ссылок
    try:
        time.sleep(1)
        soup = get_soup(url)
    except ConnectionError:
        print('конэкшон ерор')
        time.sleep(10)
        soup = cian.get_soup(url)
    
    if soup.find("div", {"data-name": "OfferUnpublished"}):
        print('объявление снято')
        continue
        
    cur_infa = infa_download(soup)
    all_infa.append(cur_infa)

In [ ]:
hrefs_real.index(url)

In [ ]:
len(all_infa)

Сохранялка. Сохраняем в формате `info_2000_5000.pickle`, где цифры означают с какого по какой файл скачали.

In [ ]:
# осторожнее с именами файлов! случайно можно перезаписать готовый :( 
with open('data/info_0_10.pickle', 'wb') as f:
    pickle.dump(hrefs_dict, f)

In [ ]:
# можно проверить всё ли ок сохранилось
with open('data/info_0_10.pickle', 'rb') as f:
    data_new = pickle.load(f)

Поглазеть на данные: 

In [ ]:
import pandas as pd
df = pd.DataFrame(all_infa)
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()